# Estudio Vid (Botritis)
## Pablo Lavín
## TFM - Master Ciencia de Datos (UC-UIMP)
## Beca JAE Intro ICU 2025
## Septiembre 2025 - Junio 2026

En este notebook se analiza el desempeño del modelo ECMWF frente a ERA5-Land durante el periodo 1981–2022.

Con el objetivo de mejorar la representatividad del modelo respecto a los datos observacionales, se realiza una corrección del sesgo de la temperatura media del modelo ECMWF.

Posteriormente los datos de temperatura se convierten en índices, en este caso, un índice compuesto de temperatura media (número de días que se supera una temperatura umbral) y de humedad relativa, con el objetivo de estudiar el riesgo de aparición de botritis, un tipo de epidemia que afecta a la vid, y es crítica durante los meses de fructificación (julio a septiembre).

Se incluyen métricas de sesgo, correlación, RMSE, ratio de varianzas y ROCSS.

## Configuración

In [1]:
# Cargamos paquetes
source("../../../scripts/setup_libraries.R")
source("../../../scripts/load_bc_functions.R")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Type 'citation("pROC")' for a citation.


Attaching package: ‘pROC’


The following objects are masked from ‘package:stats’:

    cov, smooth, var


Loading required package: rJava

Loading required package: loadeR.java

Java version 23x amd64 by N/A detected

NetCDF Java Library v4.6.0-SNAPSHOT (23 Apr 2015) loaded and ready

Loading required package: climate4R.UDG

climate4R.UDG version 0.2.6 (2023-06-26) is loaded

Please use 'citation("climate4R.UDG")' to cite this package.

loadeR version 1.8.1 (2023-06-22) is loaded


Get the latest stable version (1.8.2) using <devtools::install_github(c('SantanderMetGroup/climate4R.UDG','SantanderMetGroup/loadeR'))>

Please use 'citation("loadeR")' to cite this package.




    _______   ____  ___________________  __  ________ 
   / ___/ /  / /  |/  / __  /_  __/ __/ / / / / __  / 
  / /  / /  / / /|_/ / /_/ / / / / __/ / /_/ / /_/_/  
 / /__/ /__/ / /  / / __  / / / / /__ /___  / / \ \ 
 \___/____/_/_/  /_/_/ /_/ /_/  \___/    /_/\/   \_\ 
 
      github.com/SantanderMetGroup/climate4R



transformeR version 2.2.2 (2023-10-26) is loaded


Get the latest stable version (2.2.3) using <devtools::install_github('SantanderMetGroup/transformeR')>

Please see 'citation("transformeR")' to cite this package.

Loading required package: udunits2

udunits system database read from /vols/abedul/home/meteo/lavinp/miniforge3/envs/C4R/share/udunits/udunits2.xml

convertR version 0.3.0 (2025-07-31) is loaded


Development version may have an unexpected behaviour

  More information about the 'climate4R' ecosystem in: http://meteo.unican.es/climate4R


Attaching package: ‘convertR’


The following objects are masked from ‘package:loadeR’:

    hurs2huss, huss2hurs, tdps2hurs


Registered S3 method overwritten by 'verification':
  method    from
  lines.roc pROC

visualizeR version 1.6.4 (2023-10-26) is loaded

Please see 'citation("visualizeR")' to cite this package.

downscaleR version 3.3.4 (2023-06-22) is loaded

Please use 'citation("downscaleR")' to cite this package.

Loading requi

In [2]:
# Region de estudio

lon = c(-10, 5)
lat = c(35, 44)

# Color
color = colorRampPalette(rev(brewer.pal(n = 9, "RdYlBu")))

In [5]:
# Cargo datos: temperatura media (con sesgo y sin sesgo) y humedad relativa
tas_modelo_0 = readRDS("../../../data/compound_study_index/low_resolution/vine_botrytis_data/tas_botritis_model_vid_0.rds")
tas_modelo_1 = readRDS("../../../data/compound_study_index/low_resolution/vine_botrytis_data/tas_botritis_model_vid_1.rds")
tas_modelo_2 = readRDS("../../../data/compound_study_index/low_resolution/vine_botrytis_data/tas_botritis_model_vid_2.rds")
tas_modelo_3 = readRDS("../../../data/compound_study_index/low_resolution/vine_botrytis_data/tas_botritis_model_vid_3.rds")

tas_obs = readRDS("../../../data/compound_study_index/low_resolution/vine_botrytis_data/tas_botritis_obs_vid.rds")

bc_tas_modelo_0 = readRDS("../../../data/compound_study_index/low_resolution/vine_botrytis_data/bc_tas_botritis_vid_0.rds")
bc_tas_modelo_1 = readRDS("../../../data/compound_study_index/low_resolution/vine_botrytis_data/bc_tas_botritis_vid_1.rds")
bc_tas_modelo_2 = readRDS("../../../data/compound_study_index/low_resolution/vine_botrytis_data/bc_tas_botritis_vid_2.rds")
bc_tas_modelo_3 = readRDS("../../../data/compound_study_index/low_resolution/vine_botrytis_data/bc_tas_botritis_vid_3.rds")

hr_modelo_0 = readRDS("../../../data/compound_study_index/low_resolution/vine_botrytis_data/hr_botritis_model_vid_0.rds")
hr_modelo_1 = readRDS("../../../data/compound_study_index/low_resolution/vine_botrytis_data/hr_botritis_model_vid_1.rds")
hr_modelo_2 = readRDS("../../../data/compound_study_index/low_resolution/vine_botrytis_data/hr_botritis_model_vid_2.rds")
hr_modelo_3 = readRDS("../../../data/compound_study_index/low_resolution/vine_botrytis_data/hr_botritis_model_vid_3.rds")

hr_obs = readRDS("../../../data/compound_study_index/low_resolution/vine_botrytis_data/hr_botritis_obs_vid.rds")

# Paso los datos de bC a Kelvin
bc_tas_modelo_0 = gridArithmetics(bc_tas_modelo_0, 273.15, operator = "-")
bc_tas_modelo_1 = gridArithmetics(bc_tas_modelo_1, 273.15, operator = "-")
bc_tas_modelo_2 = gridArithmetics(bc_tas_modelo_2, 273.15, operator = "-")
bc_tas_modelo_3 = gridArithmetics(bc_tas_modelo_3, 273.15, operator = "-")

## Máscara para los datos

In [6]:
## Calculo el número de días que tmax > 22 grados (solo para la estructura del grid)
nd_obs = indexGrid(tx = tas_obs, index.code = "TXth", th = 22) %>% suppressMessages %>% suppressWarnings

## Máscara de tierra de ERA5 (es una variable más del propio reanális):
## Valores continuos entre 0 (no hay nada de tierra en ese gridbox) y 1 (todo el gridbox es tierra)
mask = loadGridData("/lustre/gmeteo/PTICLIMA/DATA/REANALYSIS/ERA5/lsm/lsm_era5.nc", var = "lsm") %>% suppressMessages %>% suppressWarnings

## Binarizo la máscara: Considero que todos los gridboxes con un valor por encima (debajo) de 0.5 son de tierra (mar)
mask.bin = binaryGrid(mask, condition = "GE", threshold = 0.5, values = c(NA, 1))

## Hago el upscaling como hice con los datos de ERA5 a la resolución de 1º del modelo
mask_upscaled = interpGrid(mask.bin,
                           new.coordinates = getGrid(tas_obs),
                           method = "bilinear") %>% suppressMessages %>% suppressWarnings

## Apoyándome en la máscara binaria, me quedo únicamente con los datos en tierra y descarto el mar
mask.bin.spain = subsetGrid(mask_upscaled, lonLim = c(-10, 5), latLim = c(35, 44))
mask.bin.spain$Data = aperm(replicate(getShape(nd_obs)["time"], mask.bin.spain$Data, simplify = "array"), c(3, 1, 2))
attributes(mask.bin.spain$Data)$dimensions = c("time", "lat", "lon")

## Máscara para el moodelo
n.members = getShape(tas_modelo_0)["member"]
mask.data = mask.bin.spain$Data
mask.4d = array(NA, dim = c(n.members, dim(mask.data)))  # member x time x lat x lon
for (m in 1:n.members) {
    mask.4d[m,,,] = mask.data
}

mask.model = mask.bin.spain  # copia de la estructura
mask.model$Data = mask.4d
attributes(mask.model$Data)$dimensions = c("member", "time", "lat", "lon")

## Fructificación (julio-septiembre). Riesgo botritis (Nº días con Tmed > 15ºC y HR > 80%)

### Representación del sesgo de la temperatura media

In [5]:
# Climatología de las obs
ref = climatology(tas_obs) %>% suppressMessages %>% suppressWarnings

# Climatología del modelos con los 4 leadtime (datos con sesgo)
diff_0 = climatology(tas_modelo_0, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff_1 = climatology(tas_modelo_1, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff_2 = climatology(tas_modelo_2, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff_3 = climatology(tas_modelo_3, by.member = FALSE) %>% suppressMessages %>% suppressWarnings

# Sesgo para cada leadtime
bias_0 = gridArithmetics(diff_0, ref, operator = "-")
bias_1 = gridArithmetics(diff_1, ref, operator = "-")
bias_2 = gridArithmetics(diff_2, ref, operator = "-")
bias_3 = gridArithmetics(diff_3, ref, operator = "-")

# Representación
b_0 = spatialPlot(bias_0, backdrop.theme = "countries", main = "Raw (lt 0)", col.regions = color, at = seq(-4, 4, 0.1))
b_1 = spatialPlot(bias_1, backdrop.theme = "countries", main = "Raw (lt 1)", col.regions = color, at = seq(-4, 4, 0.1))
b_2 = spatialPlot(bias_2, backdrop.theme = "countries", main = "Raw (lt 2)", col.regions = color, at = seq(-4, 4, 0.1))
b_3 = spatialPlot(bias_3, backdrop.theme = "countries", main = "Raw (lt 3)", col.regions = color, at = seq(-4, 4, 0.1))

# Climatología de cada leadtime (datos sin sesgo)
diff0 = climatology(bc_tas_modelo_0, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff1 = climatology(bc_tas_modelo_1, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff2 = climatology(bc_tas_modelo_2, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff3 = climatology(bc_tas_modelo_3, by.member = FALSE) %>% suppressMessages %>% suppressWarnings

# Cálculo del sesgo
bias0 = gridArithmetics(diff0, ref, operator = "-")
bias1 = gridArithmetics(diff1, ref, operator = "-")
bias2 = gridArithmetics(diff2, ref, operator = "-")
bias3 = gridArithmetics(diff3, ref, operator = "-")

# Representación
b0 = spatialPlot(bias0, backdrop.theme = "countries", main = "BC (lt 0)", col.regions = color, at = seq(-0.003, 0.003, 0.0001))
b1 = spatialPlot(bias1, backdrop.theme = "countries", main = "BC (lt 1)", col.regions = color, at = seq(-0.003, 0.003, 0.0001))
b2 = spatialPlot(bias2, backdrop.theme = "countries", main = "BC (lt 2)", col.regions = color, at = seq(-0.003, 0.003, 0.0001))
b3 = spatialPlot(bias3, backdrop.theme = "countries", main = "BC (lt 3)", col.regions = color, at = seq(-0.003, 0.003, 0.0001))

In [7]:
png("bias_tmed_fructificacion_vid.png", width = 2000, height = 1000, res = 150)
grid.arrange(b_0, b_1, b_2, b_3, b0, b1, b2, b3, ncol = 4)
dev.off()

pdf 
  2

### Funciones auxiliares

In [7]:
# Calcula la media anual de días que cumplen una condición conjunta de temperatura
# y humedad relativa a partir de datos diarios.
#
# Esta función se utilizado para datos de observaciones con dimeniones [time, lat, lon]
#
# @param tas_obs Lista con los datos diarios de temperatura y su información asociada.
# @param hr_obs Lista con los datos diarios de humedad relativa y su información asociada.
# @param temp_thresh Umbral de temperatura (por defecto 25 ºC).
# @param hr_min Humedad relativa mínima (por defecto 60%).
# @param hr_max Humedad relativa máxima (por defecto 80%).
#
# @return Lista tipo "grid" con la media anual de días que cumplen la condición.
#

compute_masked_obs = function(tas_obs, hr_obs, temp_thresh = 25, hr_min = 60, hr_max = 80) {
    
    # Vector de fechas diarias
    dates = as.Date(tas_obs$Dates$start)
    years = factor(format(dates, "%Y"))
    unique_years = levels(years)
    n_years = length(unique_years)
    nlat = dim(tas_obs$Data)[2]
    nlon = dim(tas_obs$Data)[3]
    
    # Condición conjunta: tas > temp_thresh y hr_min <= hr <= hr_max
    mask = (tas_obs$Data > temp_thresh) & (hr_obs$Data >= hr_min & hr_obs$Data <= hr_max)
    
    # Array [time(year), lat, lon]
    annual_days = array(NA, dim = c(n_years, nlat, nlon))
    
    for (y in 1:n_years) {
        idx = which(years == unique_years[y])
        annual_days[y,,] = apply(mask[idx,,], c(2,3), sum, na.rm = TRUE)
    }
    
    # Reconstruyo el grid
    grid = list()
    grid$Data = annual_days
    attr(grid$Data, "dimensions") = c("time", "lat", "lon")
    grid$xyCoords = tas_obs$xyCoords
    grid$Variable = tas_obs$Variable
    grid$Dates = tas_obs$Dates
    class(grid) = "grid"
    
    # Aplico la máscara de los datos
    grid = gridArithmetics(grid, mask.bin.spain, operator = "*")
    
    return(grid)
}

In [8]:
# Calcula la media anual de días que cumplen una condición conjunta de temperatura
# y humedad relativa para un modelo con múltiples miembros. Devuelve la media anual
# por miembro y la media sobre todos los miembros.
#
# Esta función se utilizado para datos de observaciones con dimeniones [time, lat, lon]
#
# @param tas_model Lista con los datos diarios de temperatura del modelo [member, time, lat, lon].
# @param hr_model Lista con los datos diarios de humedad relativa del modelo [member, time, lat, lon].
# @param temp_thresh Umbral de temperatura (por defecto 25 ºC).
# @param hr_min Humedad relativa mínima (por defecto 60%).
# @param hr_max Humedad relativa máxima (por defecto 80%).
#
# @return Lista con dos elementos: 
#   - per_member: grid con la media anual de días por cada miembro
#   - mean_members: grid con la media anual sobre todos los miembros

compute_masked = function(tas_model, hr_model, temp_thresh = 25, hr_min = 60, hr_max = 80) {
    
    # Dimensiones
    dims = dim(tas_model$Data)
    nmem = dims[1]
    ntime = dims[2]
    nlat = dims[3]
    nlon = dims[4]

    # Vector de fechas diarias
    dates = as.Date(tas_model$Dates$start)
    years = factor(format(dates, "%Y"))
    unique_years = levels(years)
    n_years = length(unique_years)

    # Inicializamos array para el resultado [member, year, lat, lon]
    annual_days_members = array(NA, dim = c(nmem, n_years, nlat, nlon))
    
    for(m in 1:nmem) {
        mask = (tas_model$Data[m,,,] > temp_thresh) & 
        (hr_model$Data[m,,,] >= hr_min & hr_model$Data[m,,,] <= hr_max)
    
        for(y in 1:n_years) {
            # índices de días para el año actual
            idx = which(years == unique_years[y])
            # suma diaria por píxel (número de días que cumplen condición)
            annual_days_members[m,y,,] = apply(mask[idx,,], c(2,3), sum, na.rm=TRUE)
        }
    }

    # Media entre miembros por año
    mean_over_members = apply(annual_days_members, c(2,3,4), mean, na.rm = TRUE)

    # Reconstruyo grids
    grid_members = list()
    grid_members$Data = annual_days_members
    attr(grid_members$Data, "dimensions") = c("member", "time", "lat", "lon")
    grid_members$xyCoords = tas_model$xyCoords
    grid_members$Variable = tas_model$Variable
    grid_members$Dates = tas_model$Dates
    class(grid_members) = "grid"

    grid_mean = list()
    grid_mean$Data = mean_over_members
    attr(grid_mean$Data, "dimensions") = c("time", "lat", "lon")
    grid_mean$xyCoords = tas_model$xyCoords
    grid_mean$Variable = tas_model$Variable
    grid_mean$Dates = tas_model$Dates
    class(grid_mean) = "grid"

    # Aplico la máscara de los datos con la dimensión de miembros
    grid_members = gridArithmetics(grid_members, mask.model, operator = "*")
    # Aplico la máscara de los datos sin la dimensión de miembros
    grid_mean = gridArithmetics(grid_mean, mask.bin.spain, operator = "*")

    return(list(per_member = grid_members, mean_members = grid_mean))
}

### Máscara de los datos raw

In [9]:
# Aplico la máscara donde evalúo las condiciones a los datos del modelo
grid_masked_0 = compute_masked(tas_modelo_0, hr_modelo_0, temp_thresh = 15, hr_min = 80, hr_max = 100)$mean_members
grid_masked_1 = compute_masked(tas_modelo_1, hr_modelo_1, temp_thresh = 15, hr_min = 80, hr_max = 100)$mean_members
grid_masked_2 = compute_masked(tas_modelo_2, hr_modelo_2, temp_thresh = 15, hr_min = 80, hr_max = 100)$mean_members
grid_masked_3 = compute_masked(tas_modelo_3, hr_modelo_3, temp_thresh = 15, hr_min = 80, hr_max = 100)$mean_members

In [10]:
# Representación
nd_0 = spatialPlot(climatology(grid_masked_0, by.member = FALSE), backdrop.theme = "countries",
                   main = "Raw (lt 0)", col.regions = color, at = seq(0, 67, 0.1)) %>% suppressMessages %>% suppressWarnings
nd_1 = spatialPlot(climatology(grid_masked_1, by.member = FALSE), backdrop.theme = "countries",
                   main = "Raw (lt 1)", col.regions = color, at = seq(0, 67, 0.1)) %>% suppressMessages %>% suppressWarnings
nd_2 = spatialPlot(climatology(grid_masked_2, by.member = FALSE), backdrop.theme = "countries",
                   main = "Raw (lt 2)", col.regions = color, at = seq(0, 67, 0.1)) %>% suppressMessages %>% suppressWarnings
nd_3 = spatialPlot(climatology(grid_masked_3, by.member = FALSE), backdrop.theme = "countries",
                   main = "Raw (lt 3)", col.regions = color, at = seq(0, 67, 0.1)) %>% suppressMessages %>% suppressWarnings

### Máscara de los datos bC

In [11]:
# Aplico la máscara donde evalúo las condiciones a los datos del modelo
grid_bc_masked_0 = compute_masked(bc_tas_modelo_0, hr_modelo_0, temp_thresh = 15, hr_min = 80, hr_max = 100)$mean_members
grid_bc_masked_1 = compute_masked(bc_tas_modelo_1, hr_modelo_1, temp_thresh = 15, hr_min = 80, hr_max = 100)$mean_members
grid_bc_masked_2 = compute_masked(bc_tas_modelo_2, hr_modelo_2, temp_thresh = 15, hr_min = 80, hr_max = 100)$mean_members
grid_bc_masked_3 = compute_masked(bc_tas_modelo_3, hr_modelo_3, temp_thresh = 15, hr_min = 80, hr_max = 100)$mean_members

In [12]:
# Representación
bc_nd_0 = spatialPlot(climatology(grid_bc_masked_0, by.member = FALSE), backdrop.theme = "countries",
                      main = "BC (lt 0)", col.regions = color, at = seq(0, 67, 0.1)) %>% suppressMessages %>% suppressWarnings
bc_nd_1 = spatialPlot(climatology(grid_bc_masked_1, by.member = FALSE), backdrop.theme = "countries",
                      main = "BC (lt 1)", col.regions = color, at = seq(0, 67, 0.1)) %>% suppressMessages %>% suppressWarnings
bc_nd_2 = spatialPlot(climatology(grid_bc_masked_2, by.member = FALSE), backdrop.theme = "countries",
                      main = "BC (lt 2)", col.regions = color, at = seq(0, 67, 0.1)) %>% suppressMessages %>% suppressWarnings
bc_nd_3 = spatialPlot(climatology(grid_bc_masked_3, by.member = FALSE), backdrop.theme = "countries",
                      main = "BC (lt 3)", col.regions = color, at = seq(0, 67, 0.1)) %>% suppressMessages %>% suppressWarnings

In [28]:
png("ndays_riesgo_botritis_model_vid.png", width = 2000, height = 1000, res = 150)
grid.arrange(nd_0, nd_1, nd_2, nd_3, bc_nd_0, bc_nd_1, bc_nd_2, bc_nd_3, ncol = 4)
dev.off()

pdf 
  2

### Máscara de los datos de ERA-5

In [13]:
# Aplico la máscara donde evalúo las condiciones
grid_masked_obs = compute_masked_obs(tas_obs, hr_obs, temp_thresh = 15, hr_min = 80, hr_max = 100)

# Representación
nd_obs = spatialPlot(climatology(grid_masked_obs), backdrop.theme = "countries",
                         main = "Obs", col.regions = color, at = seq(0, 67, 0.1)) %>% suppressMessages %>% suppressWarnings

In [30]:
png("ndays_riesgo_botritis_obs_vid.png", width = 2000, height = 1000, res = 150)
grid.arrange(nd_obs, ncol = 1)
dev.off()

pdf 
  2

## Bias ndays

In [18]:
# Uilizo los datos con la dimensión de miembro
grid_mem_masked_0 = compute_masked(tas_modelo_0, hr_modelo_0, temp_thresh = 15, hr_min = 80, hr_max = 100)$per_member
grid_mem_masked_1 = compute_masked(tas_modelo_1, hr_modelo_1, temp_thresh = 15, hr_min = 80, hr_max = 100)$per_member
grid_mem_masked_2 = compute_masked(tas_modelo_2, hr_modelo_2, temp_thresh = 15, hr_min = 80, hr_max = 100)$per_member
grid_mem_masked_3 = compute_masked(tas_modelo_3, hr_modelo_3, temp_thresh = 15, hr_min = 80, hr_max = 100)$per_member

bc_grid_mem_masked_0 = compute_masked(bc_tas_modelo_0, hr_modelo_0, temp_thresh = 15, hr_min = 80, hr_max = 100)$per_member
bc_grid_mem_masked_1 = compute_masked(bc_tas_modelo_1, hr_modelo_1, temp_thresh = 15, hr_min = 80, hr_max = 100)$per_member
bc_grid_mem_masked_2 = compute_masked(bc_tas_modelo_2, hr_modelo_2, temp_thresh = 15, hr_min = 80, hr_max = 100)$per_member
bc_grid_mem_masked_3 = compute_masked(bc_tas_modelo_3, hr_modelo_3, temp_thresh = 15, hr_min = 80, hr_max = 100)$per_member

### obs y raw

In [37]:
# Calculo del bias del ndays para cada leadtime del modelo
bias_tas_days_0 = veriApply(verifun = "EnsMe", 
                            fcst = grid_mem_masked_0$Data, 
                            obs = grid_masked_obs$Data, 
                            ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bias_tas_days_1 = veriApply(verifun = "EnsMe", 
                            fcst = grid_mem_masked_1$Data, 
                            obs = grid_masked_obs$Data, 
                            ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bias_tas_days_2 = veriApply(verifun = "EnsMe", 
                            fcst = grid_mem_masked_2$Data, 
                            obs = grid_masked_obs$Data, 
                            ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bias_tas_days_3 = veriApply(verifun = "EnsMe", 
                            fcst = grid_mem_masked_3$Data, 
                            obs = grid_masked_obs$Data, 
                            ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
bias_ndtas_grid_0 = easyVeri2grid(easyVeri.mat = bias_tas_days_0, obs.grid = tas_obs, verifun = "EnsMe")
bias_ndtas_grid_1 = easyVeri2grid(easyVeri.mat = bias_tas_days_1, obs.grid = tas_obs, verifun = "EnsMe")
bias_ndtas_grid_2 = easyVeri2grid(easyVeri.mat = bias_tas_days_2, obs.grid = tas_obs, verifun = "EnsMe")
bias_ndtas_grid_3 = easyVeri2grid(easyVeri.mat = bias_tas_days_3, obs.grid = tas_obs, verifun = "EnsMe")

In [42]:
# Representación
bias_tas_0 = spatialPlot(climatology(bias_ndtas_grid_0),
                         backdrop.theme = "countries",
                         col.regions = color,
                         main = "Raw (lt 0)",
                         at = seq(-25, 25, 1)) %>% suppressMessages %>% suppressWarnings

bias_tas_1 = spatialPlot(climatology(bias_ndtas_grid_1),
                         backdrop.theme = "countries",
                         col.regions = color,
                         main = "Raw (lt 1)",
                         at = seq(-25, 25, 1)) %>% suppressMessages %>% suppressWarnings

bias_tas_2 = spatialPlot(climatology(bias_ndtas_grid_2),
                         backdrop.theme = "countries",
                         col.regions = color,
                         main = "Raw (lt 2)",
                         at = seq(-25, 25, 1)) %>% suppressMessages %>% suppressWarnings

bias_tas_3 = spatialPlot(climatology(bias_ndtas_grid_3),
                         backdrop.theme = "countries",
                         col.regions = color,
                         main = "Raw (lt 3)",
                         at = seq(-25, 25, 1)) %>% suppressMessages %>% suppressWarnings

### obs y bC

In [43]:
# Calculo del bias del ndays para cada leadtime del modelo (bias corregido)
bc_bias_tas_days_0 = veriApply(verifun = "EnsMe", 
                               fcst = bc_grid_mem_masked_0$Data, 
                               obs = grid_masked_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bc_bias_tas_days_1 = veriApply(verifun = "EnsMe", 
                               fcst = bc_grid_mem_masked_1$Data, 
                               obs = grid_masked_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bc_bias_tas_days_2 = veriApply(verifun = "EnsMe", 
                               fcst = bc_grid_mem_masked_2$Data, 
                               obs = grid_masked_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bc_bias_tas_days_3 = veriApply(verifun = "EnsMe", 
                               fcst = bc_grid_mem_masked_3$Data, 
                               obs = grid_masked_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
bc_bias_ndtas_grid_0 = easyVeri2grid(easyVeri.mat = bc_bias_tas_days_0, obs.grid = tas_obs, verifun = "EnsMe")
bc_bias_ndtas_grid_1 = easyVeri2grid(easyVeri.mat = bc_bias_tas_days_1, obs.grid = tas_obs, verifun = "EnsMe")
bc_bias_ndtas_grid_2 = easyVeri2grid(easyVeri.mat = bc_bias_tas_days_2, obs.grid = tas_obs, verifun = "EnsMe")
bc_bias_ndtas_grid_3 = easyVeri2grid(easyVeri.mat = bc_bias_tas_days_3, obs.grid = tas_obs, verifun = "EnsMe")

In [44]:
# Representación
bc_bias_tas_0 = spatialPlot(climatology(bc_bias_ndtas_grid_0),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "BC (lt 0)",
                            at = seq(-25, 25, 1)) %>% suppressMessages %>% suppressWarnings

bc_bias_tas_1 = spatialPlot(climatology(bc_bias_ndtas_grid_1),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "BC (lt 1)",
                            at = seq(-25, 25, 1)) %>% suppressMessages %>% suppressWarnings

bc_bias_tas_2 = spatialPlot(climatology(bc_bias_ndtas_grid_2),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "BC (lt 2)",
                            at = seq(-25, 25, 1)) %>% suppressMessages %>% suppressWarnings

bc_bias_tas_3 = spatialPlot(climatology(bc_bias_ndtas_grid_3),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "BC (lt 3)",
                            at = seq(-25, 25, 1)) %>% suppressMessages %>% suppressWarnings

In [45]:
png("bias_ndays_botritis_vid.png", width = 2000, height = 1000, res = 150)
grid.arrange(bias_tas_0, bias_tas_1, bias_tas_2, bias_tas_3,
             bc_bias_tas_0, bc_bias_tas_1, bc_bias_tas_2, bc_bias_tas_3, ncol = 4)
dev.off()

pdf 
  2

## Correlación ndays

### test de hipótesis para puntos significativos

In [46]:
# Función para calcular correlación de Pearson y valores p entre datos de modelo y observaciones en una grilla espacial
# Además, identifica y marca los puntos con correlación estadísticamente significativa según un umbral de p-valor
#
# Args:
#   model_data: objeto con datos del modelo, estructura esperada con dimensión [miembros, tiempo, latitud, longitud]
#   obs_data: objeto con datos observacionales, estructura con dimensión [tiempo, latitud, longitud]
#   ref_grid: objeto referencia con metadatos espaciales y temporales para construir grillas (xyCoords, Variable, Dates)
#   threshold: umbral para marcar significancia estadística (p-valor), default 0.05
#
# Returns:
#   Lista con:
#     - cor: matriz de correlaciones [lat x lon]
#     - pval: matriz de valores p [lat x lon]
#     - pval_grid: objeto tipo "grid" con valores p y metadatos
#     - pts: lista de objetos para graficar puntos de significancia (stippling)

calc_cor_pval_grid = function(model_data, obs_data, ref_grid, threshold = 0.05) {
    
    # Calcular la media del ensamble para cada punto [tiempo, lat, lon]
    ens_mean = apply(model_data$Data, c(2, 3, 4), mean, na.rm = TRUE)
    
    # Dimensiones espaciales (latitud y longitud)
    lat_n = dim(ens_mean)[2]
    lon_n = dim(ens_mean)[3]
  
    # Inicializar matrices vacías para almacenar correlaciones y p-valores
    cor_array = matrix(NA, nrow = lat_n, ncol = lon_n)
    pval_array = matrix(NA, nrow = lat_n, ncol = lon_n)
    
    # Iterar sobre cada punto espacial
    for (i in 1:lat_n) {
        for (j in 1:lon_n) {
            
            # Extraer series temporales de modelo y observaciones para la celda actual
            pred_series = ens_mean[, i, j]
            obs_series = obs_data$Data[, i, j]
      
            # Filtrar índices con datos completos (no NA)
            valid_idx = complete.cases(pred_series, obs_series)
            
            # Solo calcular correlación si hay suficientes datos (mínimo 10)
            if (sum(valid_idx) >= 10) {
                test = cor.test(pred_series[valid_idx], obs_series[valid_idx], method = "pearson")
                cor_array[i, j] = test$estimate  # Coeficiente de correlación
                pval_array[i, j] = test$p.value  # Valor p de la prueba
            }
        }
    }
  
    # Construir un objeto "grid" para los valores p, con metadatos espaciales y temporales
    pval_grid = list()
    pval_grid$Data = pval_array
    attr(pval_grid$Data, "dimensions") = c("lat", "lon")
    pval_grid$xyCoords = ref_grid$xyCoords
    pval_grid$Variable = ref_grid$Variable
    pval_grid$Dates = ref_grid$Dates
    class(pval_grid) = "grid"

    pval_grid$Variable$varName = "p-values"
    attr(pval_grid$Variable, "description") = "Mapa de p-valores"
    attr(pval_grid$Variable, "units") = ""
    attr(pval_grid$Variable, "longname") = "p-values"
    
    # Crear objetos para graficar puntos de significancia estadística (stippling)
    pts = map.stippling(climatology(pval_grid), 
                        threshold = threshold, 
                        condition = "LT", 
                        pch = 19, col = "black", cex = 0.5) %>% suppressMessages() %>% suppressWarnings()
    
    # Devolver lista con resultados y objetos para plot
    return(list(cor = cor_array, pval = pval_array, pval_grid = pval_grid, pts = pts))
}

In [55]:
# Datos raw tas
test_cor_tas_0 = calc_cor_pval_grid(grid_mem_masked_0, grid_masked_obs, grid_mem_masked_0)
test_cor_tas_1 = calc_cor_pval_grid(grid_mem_masked_1, grid_masked_obs, grid_mem_masked_1)
test_cor_tas_2 = calc_cor_pval_grid(grid_mem_masked_2, grid_masked_obs, grid_mem_masked_2)
test_cor_tas_3 = calc_cor_pval_grid(grid_mem_masked_3, grid_masked_obs, grid_mem_masked_3)

# Datos bC tas
test_cor_tas_0_bc = calc_cor_pval_grid(bc_grid_mem_masked_0, grid_masked_obs, bc_grid_mem_masked_0)
test_cor_tas_1_bc = calc_cor_pval_grid(bc_grid_mem_masked_0, grid_masked_obs, bc_grid_mem_masked_0)
test_cor_tas_2_bc = calc_cor_pval_grid(bc_grid_mem_masked_0, grid_masked_obs, bc_grid_mem_masked_0)
test_cor_tas_3_bc = calc_cor_pval_grid(bc_grid_mem_masked_0, grid_masked_obs, bc_grid_mem_masked_0)

### obs y raw

In [48]:
# Calculo de correlacion para cada leadtime del modelo
cor_ndays_tas_0 = veriApply(verifun = "EnsCorr",
                            fcst = grid_mem_masked_0$Data, 
                            obs = grid_masked_obs$Data, 
                            ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

cor_ndays_tas_1 = veriApply(verifun = "EnsCorr",
                            fcst = grid_mem_masked_1$Data, 
                            obs = grid_masked_obs$Data, 
                            ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

cor_ndays_tas_2 = veriApply(verifun = "EnsCorr",
                            fcst = grid_mem_masked_2$Data, 
                            obs = grid_masked_obs$Data, 
                            ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

cor_ndays_tas_3 = veriApply(verifun = "EnsCorr",
                            fcst = grid_mem_masked_3$Data, 
                            obs = grid_masked_obs$Data, 
                            ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
cor_ndtas_grid_0 = easyVeri2grid(easyVeri.mat = cor_ndays_tas_0, obs.grid = tas_obs, verifun = "EnsCorr")
cor_ndtas_grid_1 = easyVeri2grid(easyVeri.mat = cor_ndays_tas_1, obs.grid = tas_obs, verifun = "EnsCorr")
cor_ndtas_grid_2 = easyVeri2grid(easyVeri.mat = cor_ndays_tas_2, obs.grid = tas_obs, verifun = "EnsCorr")
cor_ndtas_grid_3 = easyVeri2grid(easyVeri.mat = cor_ndays_tas_3, obs.grid = tas_obs, verifun = "EnsCorr")

In [52]:
# Representación
c0_tas = spatialPlot(climatology(cor_ndtas_grid_0),
                     backdrop.theme = "countries",
                     sp.layout = list(test_cor_tas_0$pts),
                     col.regions = color,
                     at = seq(-1, 1, 0.1),
                     main = "Raw (lt 0)") %>% suppressMessages %>% suppressWarnings

c1_tas = spatialPlot(climatology(cor_ndtas_grid_1),
                     backdrop.theme = "countries",
                     sp.layout = list(test_cor_tas_1$pts),
                     col.regions = color,
                     at = seq(-1, 1, 0.1),
                     main = "Raw (lt 1)") %>% suppressMessages %>% suppressWarnings

c2_tas = spatialPlot(climatology(cor_ndtas_grid_2),
                     backdrop.theme = "countries",
                     sp.layout = list(test_cor_tas_2$pts),
                     col.regions = color,
                     at = seq(-1, 1, 0.1),
                     main = "Raw (lt 2)") %>% suppressMessages %>% suppressWarnings

c3_tas = spatialPlot(climatology(cor_ndtas_grid_3),
                     backdrop.theme = "countries",
                     sp.layout = list(test_cor_tas_3$pts),
                     col.regions = color,
                     at = seq(-1, 1, 0.1),
                     main = "Raw (lt 3)") %>% suppressMessages %>% suppressWarnings

### obs y bC

In [51]:
# Calculo de correlacion para cada leadtime del modelo
bc_cor_ndays_tas_0 = veriApply(verifun = "EnsCorr",
                               fcst = bc_grid_mem_masked_0$Data, 
                               obs = grid_masked_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Calculo de correlacion para cada leadtime del modelo
bc_cor_ndays_tas_1 = veriApply(verifun = "EnsCorr",
                               fcst = bc_grid_mem_masked_1$Data, 
                               obs = grid_masked_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Calculo de correlacion para cada leadtime del modelo
bc_cor_ndays_tas_2 = veriApply(verifun = "EnsCorr",
                               fcst = bc_grid_mem_masked_2$Data, 
                               obs = grid_masked_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Calculo de correlacion para cada leadtime del modelo
bc_cor_ndays_tas_3 = veriApply(verifun = "EnsCorr",
                               fcst = bc_grid_mem_masked_3$Data, 
                               obs = grid_masked_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
bc_cor_ndtas_grid_0 = easyVeri2grid(easyVeri.mat = bc_cor_ndays_tas_0, obs.grid = tas_obs, verifun = "EnsCorr")
bc_cor_ndtas_grid_1 = easyVeri2grid(easyVeri.mat = bc_cor_ndays_tas_1, obs.grid = tas_obs, verifun = "EnsCorr")
bc_cor_ndtas_grid_2 = easyVeri2grid(easyVeri.mat = bc_cor_ndays_tas_2, obs.grid = tas_obs, verifun = "EnsCorr")
bc_cor_ndtas_grid_3 = easyVeri2grid(easyVeri.mat = bc_cor_ndays_tas_3, obs.grid = tas_obs, verifun = "EnsCorr")

In [53]:
# Representación
bc_c0_tas = spatialPlot(climatology(bc_cor_ndtas_grid_0),
                        backdrop.theme = "countries",
                        sp.layout = list(test_cor_tas_0_bc$pts),
                        col.regions = color,
                        at = seq(-1, 1, 0.1),
                        main = "BC (lt 0)") %>% suppressMessages %>% suppressWarnings

bc_c1_tas = spatialPlot(climatology(bc_cor_ndtas_grid_1),
                        backdrop.theme = "countries",
                        sp.layout = list(test_cor_tas_1_bc$pts),
                        col.regions = color,
                        at = seq(-1, 1, 0.1),
                        main = "BC (lt 1)") %>% suppressMessages %>% suppressWarnings

bc_c2_tas = spatialPlot(climatology(bc_cor_ndtas_grid_2),
                        backdrop.theme = "countries",
                        sp.layout = list(test_cor_tas_2_bc$pts),
                        col.regions = color,
                        at = seq(-1, 1, 0.1),
                        main = "BC (lt 2)") %>% suppressMessages %>% suppressWarnings

bc_c3_tas = spatialPlot(climatology(bc_cor_ndtas_grid_3),
                        backdrop.theme = "countries",
                        sp.layout = list(test_cor_tas_3_bc$pts),
                        col.regions = color,
                        at = seq(-1, 1, 0.1),
                        main = "BC (lt 3)") %>% suppressMessages %>% suppressWarnings

In [54]:
png("cor_ndays_botritis_vid.png", width = 2000, height = 1000, res = 150)
grid.arrange(c0_tas, c1_tas, c2_tas, c3_tas, bc_c0_tas, bc_c1_tas, bc_c2_tas, bc_c3_tas, ncol = 4)
dev.off()

pdf 
  2

## RMSE ndays

### obs y raw

In [56]:
# Calculo del RMSE del ndays para cada leadtime del modelo
rmse_tas_days_0 = veriApply(verifun = "EnsRmse", 
                            fcst = grid_mem_masked_0$Data, 
                            obs = grid_masked_obs$Data, 
                            ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rmse_tas_days_1 = veriApply(verifun = "EnsRmse", 
                            fcst = grid_mem_masked_1$Data, 
                            obs = grid_masked_obs$Data, 
                            ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rmse_tas_days_2 = veriApply(verifun = "EnsRmse", 
                            fcst = grid_mem_masked_2$Data, 
                            obs = grid_masked_obs$Data, 
                            ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rmse_tas_days_3 = veriApply(verifun = "EnsRmse", 
                            fcst = grid_mem_masked_3$Data, 
                            obs = grid_masked_obs$Data, 
                            ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
rmse_ndtas_grid_0 = easyVeri2grid(easyVeri.mat = rmse_tas_days_0, obs.grid = tas_obs, verifun = "EnsRmse")
rmse_ndtas_grid_1 = easyVeri2grid(easyVeri.mat = rmse_tas_days_1, obs.grid = tas_obs, verifun = "EnsRmse")
rmse_ndtas_grid_2 = easyVeri2grid(easyVeri.mat = rmse_tas_days_2, obs.grid = tas_obs, verifun = "EnsRmse")
rmse_ndtas_grid_3 = easyVeri2grid(easyVeri.mat = rmse_tas_days_3, obs.grid = tas_obs, verifun = "EnsRmse")

In [68]:
# Representación RMSE
rmse_tas_0 = spatialPlot(climatology(rmse_ndtas_grid_0),
                         backdrop.theme = "countries",
                         col.regions = color,
                         main = "Raw (lt 0)",
                         at = seq(0, 30, 1)) %>% suppressMessages %>% suppressWarnings

rmse_tas_1 = spatialPlot(climatology(rmse_ndtas_grid_1),
                         backdrop.theme = "countries",
                         col.regions = color,
                         main = "Raw (lt 1)",
                         at = seq(0, 30, 1)) %>% suppressMessages %>% suppressWarnings

rmse_tas_2 = spatialPlot(climatology(rmse_ndtas_grid_2),
                         backdrop.theme = "countries",
                         col.regions = color,
                         main = "Raw (lt 2)",
                         at = seq(0, 30, 1)) %>% suppressMessages %>% suppressWarnings

rmse_tas_3 = spatialPlot(climatology(rmse_ndtas_grid_3),
                         backdrop.theme = "countries",
                         col.regions = color,
                         main = "Raw (lt 3)",
                         at = seq(0, 30, 1)) %>% suppressMessages %>% suppressWarnings

### obs y bC

In [62]:
# Calculo del RMSE del ndays para cada leadtime del modelo
bc_rmse_tas_days_0 = veriApply(verifun = "EnsRmse", 
                               fcst = bc_grid_mem_masked_0$Data, 
                               obs = grid_masked_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bc_rmse_tas_days_1 = veriApply(verifun = "EnsRmse", 
                               fcst = bc_grid_mem_masked_1$Data, 
                               obs = grid_masked_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bc_rmse_tas_days_2 = veriApply(verifun = "EnsRmse", 
                               fcst = bc_grid_mem_masked_2$Data, 
                               obs = grid_masked_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bc_rmse_tas_days_3 = veriApply(verifun = "EnsRmse", 
                               fcst = bc_grid_mem_masked_3$Data, 
                               obs = grid_masked_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
bc_rmse_ndtas_grid_0 = easyVeri2grid(easyVeri.mat = bc_rmse_tas_days_0, obs.grid = tas_obs, verifun = "EnsRmse")
bc_rmse_ndtas_grid_1 = easyVeri2grid(easyVeri.mat = bc_rmse_tas_days_1, obs.grid = tas_obs, verifun = "EnsRmse")
bc_rmse_ndtas_grid_2 = easyVeri2grid(easyVeri.mat = bc_rmse_tas_days_2, obs.grid = tas_obs, verifun = "EnsRmse")
bc_rmse_ndtas_grid_3 = easyVeri2grid(easyVeri.mat = bc_rmse_tas_days_3, obs.grid = tas_obs, verifun = "EnsRmse")

In [69]:
# Representación
bc_rmse_tas_0 = spatialPlot(climatology(bc_rmse_ndtas_grid_0),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "BC (lt 0)",
                            at = seq(0, 30, 1)) %>% suppressMessages %>% suppressWarnings

bc_rmse_tas_1 = spatialPlot(climatology(bc_rmse_ndtas_grid_1),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "BC (lt 1)",
                            at = seq(0, 30, 1)) %>% suppressMessages %>% suppressWarnings

bc_rmse_tas_2 = spatialPlot(climatology(bc_rmse_ndtas_grid_2),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "BC (lt 2)",
                            at = seq(0, 30, 1)) %>% suppressMessages %>% suppressWarnings

bc_rmse_tas_3 = spatialPlot(climatology(bc_rmse_ndtas_grid_3),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "BC (lt 3)",
                            at = seq(0, 30, 1)) %>% suppressMessages %>% suppressWarnings

In [72]:
png("rmse_ndays_botritis_vid.png", width = 2000, height = 1000, res = 150)

grid.arrange(rmse_tas_0, rmse_tas_1, rmse_tas_2, rmse_tas_3,
             bc_rmse_tas_0, bc_rmse_tas_1, bc_rmse_tas_2, bc_rmse_tas_3, ncol = 4)
dev.off()

pdf 
  2

## Ratio varianzas ndays

### obs y raw

In [80]:
# Varianzas, calculo la varianza de cada miembro y luego hago el promedio
var_tas_0 = climatology(grid_mem_masked_0,
                        clim.fun = list(FUN = "var", na.rm = TRUE),
                        by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_tas_0 = climatology(var_tas_0,
                             clim.fun = list(FUN = "mean", na.rm = TRUE),
                             by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_tas_1 = climatology(grid_mem_masked_1,
                        clim.fun = list(FUN = "var", na.rm = TRUE),
                        by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_tas_1 = climatology(var_tas_1,
                             clim.fun = list(FUN = "mean", na.rm = TRUE),
                             by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_tas_2 = climatology(grid_mem_masked_2,
                        clim.fun = list(FUN = "var", na.rm = TRUE),
                        by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_tas_2 = climatology(var_tas_2,
                             clim.fun = list(FUN = "mean", na.rm = TRUE),
                             by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_tas_3 = climatology(grid_mem_masked_3,
                        clim.fun = list(FUN = "var", na.rm = TRUE),
                        by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_tas_3 = climatology(var_tas_3,
                             clim.fun = list(FUN = "mean", na.rm = TRUE),
                             by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_tas_obs = climatology(grid_masked_obs,
                          clim.fun = list(FUN = "var", na.rm = TRUE)) %>% suppressMessages %>% suppressWarnings

# Ratio de varianzas
rv_tas_0 = gridArithmetics(mean_var_tas_0, var_tas_obs, operator = "/")
rv_tas_1 = gridArithmetics(mean_var_tas_1, var_tas_obs, operator = "/")
rv_tas_2 = gridArithmetics(mean_var_tas_2, var_tas_obs, operator = "/")
rv_tas_3 = gridArithmetics(mean_var_tas_3, var_tas_obs, operator = "/")

# Representación
rv0_tas = spatialPlot(rv_tas_0, backdrop.theme = "countries", col.regions = color, main = "Raw (lt 0)", at = seq(0, 3, 0.1))
rv1_tas = spatialPlot(rv_tas_1, backdrop.theme = "countries", col.regions = color, main = "Raw (lt 1)", at = seq(0, 3, 0.1))
rv2_tas = spatialPlot(rv_tas_2, backdrop.theme = "countries", col.regions = color, main = "Raw (lt 2)", at = seq(0, 3, 0.1))
rv3_tas = spatialPlot(rv_tas_3, backdrop.theme = "countries", col.regions = color, main = "Raw (lt 3)", at = seq(0, 3, 0.1))

### obs y bC

In [81]:
# Varianzas, calculo la varianza de cada miembro y luego hago el promedio
bc_var_tas_0 = climatology(bc_grid_mem_masked_0,
                           clim.fun = list(FUN = "var", na.rm = TRUE),
                           by.member = TRUE) %>% suppressMessages %>% suppressWarnings

bc_mean_var_tas_0 = climatology(bc_var_tas_0,
                                clim.fun = list(FUN = "mean", na.rm = TRUE),
                                by.member = FALSE) %>% suppressMessages %>% suppressWarnings

bc_var_tas_1 = climatology(bc_grid_mem_masked_1,
                           clim.fun = list(FUN = "var", na.rm = TRUE),
                           by.member = TRUE) %>% suppressMessages %>% suppressWarnings

bc_mean_var_tas_1 = climatology(bc_var_tas_1,
                                clim.fun = list(FUN = "mean", na.rm = TRUE),
                                by.member = FALSE) %>% suppressMessages %>% suppressWarnings

bc_var_tas_2 = climatology(bc_grid_mem_masked_2,
                           clim.fun = list(FUN = "var", na.rm = TRUE),
                           by.member = TRUE) %>% suppressMessages %>% suppressWarnings

bc_mean_var_tas_2 = climatology(bc_var_tas_2,
                                clim.fun = list(FUN = "mean", na.rm = TRUE),
                                by.member = FALSE) %>% suppressMessages %>% suppressWarnings

bc_var_tas_3 = climatology(bc_grid_mem_masked_3,
                           clim.fun = list(FUN = "var", na.rm = TRUE),
                           by.member = TRUE) %>% suppressMessages %>% suppressWarnings

bc_mean_var_tas_3 = climatology(bc_var_tas_3,
                                clim.fun = list(FUN = "mean", na.rm = TRUE),
                                by.member = FALSE) %>% suppressMessages %>% suppressWarnings

# Ratio de varianzas
bc_rv_tas_0 = gridArithmetics(bc_mean_var_tas_0, var_tas_obs, operator = "/")
bc_rv_tas_1 = gridArithmetics(bc_mean_var_tas_1, var_tas_obs, operator = "/")
bc_rv_tas_2 = gridArithmetics(bc_mean_var_tas_2, var_tas_obs, operator = "/")
bc_rv_tas_3 = gridArithmetics(bc_mean_var_tas_3, var_tas_obs, operator = "/")

# Representación
bc_rv0_tas = spatialPlot(bc_rv_tas_0, backdrop.theme = "countries", col.regions = color, main = "BC (lt 0)", at = seq(0, 3, 0.1))
bc_rv1_tas = spatialPlot(bc_rv_tas_1, backdrop.theme = "countries", col.regions = color, main = "BC (lt 1)", at = seq(0, 3, 0.1))
bc_rv2_tas = spatialPlot(bc_rv_tas_2, backdrop.theme = "countries", col.regions = color, main = "BC (lt 2)", at = seq(0, 3, 0.1))
bc_rv3_tas = spatialPlot(bc_rv_tas_3, backdrop.theme = "countries", col.regions = color, main = "BC (lt 3)", at = seq(0, 3, 0.1))

In [82]:
png("ratiovars_ndays_botritis_vid.png", width = 2000, height = 1000, res = 150)
grid.arrange(rv0_tas, rv1_tas, rv2_tas, rv3_tas,
             bc_rv0_tas, bc_rv1_tas, bc_rv2_tas, bc_rv3_tas, ncol = 4)
dev.off()

pdf 
  2

## ROCSS ndays

### Funciones auxiliares

In [14]:
## Función para realizar bootstrap y calcular ROCSS
##
## Esta función calcula el ROC Skill Score (ROCSS) mediante un enfoque bootstrap,
## generando múltiples muestras re-muestreadas con reemplazo sobre la dimensión temporal.
##
## Argumentos:
## - nd_modelo: objeto con los pronósticos originales (formato esperado: [member, time, lat, lon])
## - nd_obs: objeto con las observaciones correspondientes (formato: [time, lat, lon])
## - n_boot: número de muestras bootstrap a generar (por defecto: 1000)
##
## Valor:
## - Lista de 3 arrays (uno por tercil), cada uno con dimensiones [n_boot, lat, lon],
##   que contiene los valores bootstrap del ROCSS para cada tercil.

rocss_bootstrap = function(nd_modelo, nd_obs, n_boot = 1000) {
    
    # Obtener dimensiones del array de pronóstico
    dims = dim(nd_modelo$Data)
    
    # Inicializar lista para guardar ROCSS bootstrap para los tres terciles
    rocss_bootstrap = vector("list", length = 3)
    
    # Extraer observaciones
    obs = nd_obs$Data
    
    # Inicializar arrays vacíos para cada tercil: [n_boot, lat, lon]
    for (i in 1:3) {
        rocss_bootstrap[[i]] = array(NA, dim = c(n_boot, dims[3], dims[4]))
    }
    
    # Bucle principal de bootstrap
    for (b in 1:n_boot) {
        
        # Muestreo con reemplazo sobre la dimensión temporal (índice de tiempo)
        time_idx = sample(1:dims[2], size = dims[2], replace = TRUE)
    
        # Crear pronóstico re-muestreado
        fcst_boot = nd_modelo$Data[, time_idx, , , drop = FALSE]
    
        # Calcular ROCSS con veriApply usando la función EnsRocss
        boot_result = veriApply(
            verifun = "EnsRocss",
            fcst = fcst_boot,
            obs = obs,
            prob = c(1/3, 2/3),
            ensdim = 1,   # Dimensión de los miembros del ensamble
            tdim = 2      # Dimensión temporal
        )
    
        # Guardar el resultado para cada tercil en la lista correspondiente
        for (i in 1:3) {
            rocss_bootstrap[[i]][b, , ] = boot_result[[i]]
        }
    }
    
    # Devolver lista con resultados bootstrap por tercil
    return(rocss_bootstrap)
}

In [16]:
## Función para generar capas de "stippling" que indican puntos con ROCSS significativamente mayores que un percentil dado
##
## Parámetros:
## mg               : objeto con datos ROCSS con dimensiones [tercil, member, time, lat, lon]
## rocss_bootstrap  : lista de arrays bootstrap con dimensiones [n_boot, lat, lon] para cada tercil
## ref_grid         : grid de referencia que contiene Dates y xyCoords (coordenadas XY)
## threshold        : percentil del bootstrap para determinar significancia (default 0.95)
##
## Retorna:
## lista de objetos "sp.points" para cada tercil con puntos significativos (stippling)

get_rocss_stippling_layers = function(mg, rocss_bootstrap, ref_grid, threshold = 0.95) {
	
	rocss_orig = mg$Data
	n_terciles = dim(rocss_orig)[1]
	n_lat = dim(rocss_orig)[4]
	n_lon = dim(rocss_orig)[5]
	
	xyCoords = ref_grid$xyCoords
	Dates = ref_grid$Dates
	
	stippling_list = vector("list", n_terciles)
	
	for (t in 1:n_terciles) {
		signif_mask = matrix(NA, nrow = n_lat, ncol = n_lon)
		
		for (i in 1:n_lat) {
			for (j in 1:n_lon) {
				boot_vals = rocss_bootstrap[[t]][, i, j]
				val_orig = rocss_orig[t, 1, 1, i, j]
				if (all(is.na(boot_vals)) || is.na(val_orig)) next
				q95 = quantile(boot_vals, threshold, na.rm = TRUE)
				signif_mask[i, j] = val_orig > q95
			}
		}
		
		# Construcción manual del objeto grid para la capa de significancia
		pval_grid = list()
		pval_grid$Data = signif_mask
		attr(pval_grid$Data, "dimensions") = c("lat", "lon")
		pval_grid$Dates = Dates
		pval_grid$xyCoords = xyCoords
		pval_grid$Variable = list(varName = paste0("significance_tercil_", t))
		class(pval_grid) = "grid"
		
		# Genera puntos de significancia con map.stippling (pch=19, color negro, tamaño 0.5)
		stippling_list[[t]] = suppressWarnings(
			suppressMessages(
				map.stippling(climatology(pval_grid), threshold = 0.5, condition = "GT",
				              pch = 19, col = "black", cex = 0.5)
			)
		)
	}
	
	# Elimina entradas NULL (si algún tercil no tiene puntos significativos)
	stippling_list = Filter(Negate(is.null), stippling_list)
	
	# Filtra para quedarse solo con listas que son puntos espaciales (sp.points)
	stippling_list = Filter(function(x) {
		is.list(x) && length(x) > 1 && x[[1]] == "sp.points"
	}, stippling_list)
	
	return(stippling_list)
}

### obs y raw

In [19]:
# Calculo del ROCSS para cada leadtime del modelo
rocss_ndays_tas_0 = veriApply(verifun = "EnsRocss", 
                              fcst = grid_mem_masked_0$Data, 
                              obs = grid_masked_obs$Data,
                              prob = c(1/3, 2/3),
                              ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rocss_ndays_tas_1 = veriApply(verifun = "EnsRocss", 
                              fcst = grid_mem_masked_1$Data, 
                              obs = grid_masked_obs$Data,
                              prob = c(1/3, 2/3),
                              ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rocss_ndays_tas_2 = veriApply(verifun = "EnsRocss", 
                              fcst = grid_mem_masked_2$Data, 
                              obs = grid_masked_obs$Data,
                              prob = c(1/3, 2/3),
                              ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rocss_ndays_tas_3 = veriApply(verifun = "EnsRocss", 
                              fcst = grid_mem_masked_3$Data, 
                              obs = grid_masked_obs$Data,
                              prob = c(1/3, 2/3),
                              ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Multigrid para representar los tres percentiles
mg_tas_0 = makeMultiGrid(lapply(rocss_ndays_tas_0[1:3], "easyVeri2grid", grid_masked_obs))
mg_tas_1 = makeMultiGrid(lapply(rocss_ndays_tas_1[1:3], "easyVeri2grid", grid_masked_obs))
mg_tas_2 = makeMultiGrid(lapply(rocss_ndays_tas_2[1:3], "easyVeri2grid", grid_masked_obs))
mg_tas_3 = makeMultiGrid(lapply(rocss_ndays_tas_3[1:3], "easyVeri2grid", grid_masked_obs))

In [86]:
set.seed(123)  # Reproducibilidad

# Aplico función de bootstraping
rocss_bootstrap_tas_0 = rocss_bootstrap(grid_mem_masked_0, grid_masked_obs)
rocss_bootstrap_tas_1 = rocss_bootstrap(grid_mem_masked_1, grid_masked_obs)
rocss_bootstrap_tas_2 = rocss_bootstrap(grid_mem_masked_2, grid_masked_obs)
rocss_bootstrap_tas_3 = rocss_bootstrap(grid_mem_masked_3, grid_masked_obs)

In [87]:
saveRDS(rocss_bootstrap_tas_0, file = "rocss_bootstrap_tas_raw_vid_botritis_0.rds")
saveRDS(rocss_bootstrap_tas_1, file = "rocss_bootstrap_tas_raw_vid_botritis_1.rds")
saveRDS(rocss_bootstrap_tas_2, file = "rocss_bootstrap_tas_raw_vid_botritis_2.rds")
saveRDS(rocss_bootstrap_tas_3, file = "rocss_bootstrap_tas_raw_vid_botritis_3.rds")

In [92]:
rocss_bootstrap_tas_0 = readRDS("../../../data/compound_study_index/low_resolution/vine_botrytis_data/rocss_bootstrap_tas_raw_vid_botritis_0.rds")
rocss_bootstrap_tas_1 = readRDS("../../../data/compound_study_index/low_resolution/vine_botrytis_data/rocss_bootstrap_tas_raw_vid_botritis_1.rds")
rocss_bootstrap_tas_2 = readRDS("../../../data/compound_study_index/low_resolution/vine_botrytis_data/rocss_bootstrap_tas_raw_vid_botritis_2.rds")
rocss_bootstrap_tas_3 = readRDS("../../../data/compound_study_index/low_resolution/vine_botrytis_data/rocss_bootstrap_tas_raw_vid_botritis_3.rds")

In [88]:
# Aplico función de máscara significativa
pts_layers_tas_0 = get_rocss_stippling_layers(mg_tas_0, rocss_bootstrap_tas_0, grid_mem_masked_0)
pts_layers_tas_1 = get_rocss_stippling_layers(mg_tas_1, rocss_bootstrap_tas_1, grid_mem_masked_0)
pts_layers_tas_2 = get_rocss_stippling_layers(mg_tas_2, rocss_bootstrap_tas_2, grid_mem_masked_0)
pts_layers_tas_3 = get_rocss_stippling_layers(mg_tas_3, rocss_bootstrap_tas_3, grid_mem_masked_0)

In [89]:
rocss_tas_0 = spatialPlot(climatology(mg_tas_0),
                          backdrop.theme = "countries",
                          names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                          layout = c(3,1),
                          col.regions = color,
                          at = seq(-1, 1, 0.05),
                          sp.layout = list(pts_layers_tas_0),
                          main = "Raw (lt 0)") %>% suppressMessages %>% suppressWarnings

rocss_tas_1 = spatialPlot(climatology(mg_tas_1),
                          backdrop.theme = "countries",
                          names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                          layout = c(3,1),
                          col.regions = color,
                          at = seq(-1, 1, 0.05),
                          sp.layout = list(pts_layers_tas_1),
                          main = "Raw (lt 1)") %>% suppressMessages %>% suppressWarnings

rocss_tas_2 = spatialPlot(climatology(mg_tas_2),
                          backdrop.theme = "countries",
                          names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                          layout = c(3,1),
                          col.regions = color,
                          at = seq(-1, 1, 0.05),
                          sp.layout = list(pts_layers_tas_2),
                          main = "Raw (lt 2)") %>% suppressMessages %>% suppressWarnings

rocss_tas_3 = spatialPlot(climatology(mg_tas_3),
                          backdrop.theme = "countries",
                          names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                          layout = c(3,1),
                          col.regions = color,
                          at = seq(-1, 1, 0.05),
                          sp.layout = list(pts_layers_tas_3),
                          main = "Raw (lt 3)") %>% suppressMessages %>% suppressWarnings

In [90]:
png("rocss_ndays_botritis_raw_vid.png", width = 2000, height = 1000, res = 150)
grid.arrange(rocss_tas_0, rocss_tas_1, rocss_tas_2, rocss_tas_3, ncol = 2)
dev.off()

pdf 
  2

### obs y bC

In [91]:
# Calculo del ROCSS para cada leadtime del modelo
bc_rocss_ndays_tas_0 = veriApply(verifun = "EnsRocss", 
                                 fcst = bc_grid_mem_masked_0$Data, 
                                 obs = grid_masked_obs$Data,
                                 prob = c(1/3, 2/3),
                                 ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bc_rocss_ndays_tas_1 = veriApply(verifun = "EnsRocss", 
                                 fcst = bc_grid_mem_masked_1$Data, 
                                 obs = grid_masked_obs$Data,
                                 prob = c(1/3, 2/3),
                                 ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bc_rocss_ndays_tas_2 = veriApply(verifun = "EnsRocss", 
                                 fcst = bc_grid_mem_masked_2$Data, 
                                 obs = grid_masked_obs$Data,
                                 prob = c(1/3, 2/3),
                                 ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bc_rocss_ndays_tas_3 = veriApply(verifun = "EnsRocss", 
                                 fcst = bc_grid_mem_masked_3$Data, 
                                 obs = grid_masked_obs$Data,
                                 prob = c(1/3, 2/3),
                                 ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Multigrid para representar los tres percentiles
bc_mg_tas_0 = makeMultiGrid(lapply(bc_rocss_ndays_tas_0[1:3], "easyVeri2grid", grid_masked_obs))
bc_mg_tas_1 = makeMultiGrid(lapply(bc_rocss_ndays_tas_1[1:3], "easyVeri2grid", grid_masked_obs))
bc_mg_tas_2 = makeMultiGrid(lapply(bc_rocss_ndays_tas_2[1:3], "easyVeri2grid", grid_masked_obs))
bc_mg_tas_3 = makeMultiGrid(lapply(bc_rocss_ndays_tas_3[1:3], "easyVeri2grid", grid_masked_obs))

In [93]:
# Aplico función de bootstraping
bc_rocss_bootstrap_tas_0 = rocss_bootstrap(bc_grid_mem_masked_0, grid_masked_obs)
bc_rocss_bootstrap_tas_1 = rocss_bootstrap(bc_grid_mem_masked_1, grid_masked_obs)
bc_rocss_bootstrap_tas_2 = rocss_bootstrap(bc_grid_mem_masked_2, grid_masked_obs)
bc_rocss_bootstrap_tas_3 = rocss_bootstrap(bc_grid_mem_masked_3, grid_masked_obs)

In [94]:
saveRDS(bc_rocss_bootstrap_tas_0, file = "rocss_bootstrap_tas_bc_vid_botritis_0.rds")
saveRDS(bc_rocss_bootstrap_tas_1, file = "rocss_bootstrap_tas_bc_vid_botritis_1.rds")
saveRDS(bc_rocss_bootstrap_tas_2, file = "rocss_bootstrap_tas_bc_vid_botritis_2.rds")
saveRDS(bc_rocss_bootstrap_tas_3, file = "rocss_bootstrap_tas_bc_vid_botritis_3.rds")

In [98]:
bc_rocss_bootstrap_tas_0 = readRDS("../../../data/compound_study_index/low_resolution/vine_botrytis_data/rocss_bootstrap_tas_bc_vid_botritis_0.rds")
bc_rocss_bootstrap_tas_1 = readRDS("../../../data/compound_study_index/low_resolution/vine_botrytis_data/rocss_bootstrap_tas_bc_vid_botritis_1.rds")
bc_rocss_bootstrap_tas_2 = readRDS("../../../data/compound_study_index/low_resolution/vine_botrytis_data/rocss_bootstrap_tas_bc_vid_botritis_2.rds")
bc_rocss_bootstrap_tas_3 = readRDS("../../../data/compound_study_index/low_resolution/vine_botrytis_data/rocss_bootstrap_tas_bc_vid_botritis_3.rds")

In [95]:
# Aplico función de máscara significativa
bc_pts_layers_tas_0 = get_rocss_stippling_layers(bc_mg_tas_0, bc_rocss_bootstrap_tas_0, bc_grid_mem_masked_0)
bc_pts_layers_tas_1 = get_rocss_stippling_layers(bc_mg_tas_1, bc_rocss_bootstrap_tas_1, bc_grid_mem_masked_0)
bc_pts_layers_tas_2 = get_rocss_stippling_layers(bc_mg_tas_2, bc_rocss_bootstrap_tas_2, bc_grid_mem_masked_0)
bc_pts_layers_tas_3 = get_rocss_stippling_layers(bc_mg_tas_3, bc_rocss_bootstrap_tas_3, bc_grid_mem_masked_0)

In [96]:
bc_rocss_tas_0 = spatialPlot(climatology(bc_mg_tas_0),
                          backdrop.theme = "countries",
                          names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                          layout = c(3,1),
                          col.regions = color,
                          at = seq(-1, 1, 0.05),
                          sp.layout = list(bc_pts_layers_tas_0),
                          main = "BC (lt 0)") %>% suppressMessages %>% suppressWarnings

bc_rocss_tas_1 = spatialPlot(climatology(bc_mg_tas_1),
                          backdrop.theme = "countries",
                          names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                          layout = c(3,1),
                          col.regions = color,
                          at = seq(-1, 1, 0.05),
                          sp.layout = list(bc_pts_layers_tas_1),
                          main = "BC (lt 1)") %>% suppressMessages %>% suppressWarnings

bc_rocss_tas_2 = spatialPlot(climatology(bc_mg_tas_2),
                          backdrop.theme = "countries",
                          names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                          layout = c(3,1),
                          col.regions = color,
                          at = seq(-1, 1, 0.05),
                          sp.layout = list(bc_pts_layers_tas_2),
                          main = "BC (lt 2)") %>% suppressMessages %>% suppressWarnings

bc_rocss_tas_3 = spatialPlot(climatology(bc_mg_tas_3),
                          backdrop.theme = "countries",
                          names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                          layout = c(3,1),
                          col.regions = color,
                          at = seq(-1, 1, 0.05),
                          sp.layout = list(bc_pts_layers_tas_3),
                          main = "BC (lt 3)") %>% suppressMessages %>% suppressWarnings

In [97]:
png("rocss_ndays_botritis_bc_vid.png", width = 2000, height = 1000, res = 150)
grid.arrange(bc_rocss_tas_0, bc_rocss_tas_1, bc_rocss_tas_2, bc_rocss_tas_3, ncol = 2)
dev.off()

pdf 
  2